<a href="https://colab.research.google.com/github/agemagician/CodeTrans/blob/main/prediction/single%20task/source%20code%20summarization/python/small_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install the library and download the pretrained models

In [1]:
print("Installing dependencies...")
%tensorflow_version 2.x
!pip install -q t5==0.6.4

import functools
import os
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds

import t5

!wget "https://www.dropbox.com/sh/kjoqdpj7e16dny9/AADdvjWVFckCgNQN-AqMKhiDa?dl=1" -O vocabulary.zip
!unzip vocabulary.zip
!rm vocabulary.zip
!wget "https://www.dropbox.com/sh/hvwq9obcmm0rzzi/AADCz0JpvHHVoWWSvhyTrWoAa?dl=1" -O python.zip
!unzip python.zip
!rm python.zip

Installing dependencies...
     |████████████████████████████████| 163kB 4.5MB/s 
     |████████████████████████████████| 3.6MB 14.5MB/s 
     |████████████████████████████████| 348kB 45.5MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
     |████████████████████████████████| 1.1MB 48.0MB/s 
     |████████████████████████████████| 1.3MB 47.4MB/s 
     |████████████████████████████████| 2.6MB 43.0MB/s 
     |████████████████████████████████| 890kB 46.5MB/s 
     |████████████████████████████████| 2.9MB 46.5MB/s 
ERROR: transformers 3.5.0 has requirement sentencepiece==0.1.91, but you'll have sentencepiece 0.1.94 which is incompatible.
INFO:tensorflow:tokens_length=568 inputs_length=512 targets_length=114 noise_density=0.15 mean_noise_span_length=3.0 
--2020-11-10 14:42:14--  https://www.dropbox.com/sh/kjoqdpj7e16dny9/AADdvjWVFckCgNQN-AqMKhiDa?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.c

## Set sentencepiece model

In [2]:
from t5.data.sentencepiece_vocabulary import SentencePieceVocabulary

vocab_model_path = 'code_spm_unigram_40M.model'
vocab = SentencePieceVocabulary(vocab_model_path, extra_ids=100)

print("Vocab has a size of %d\n" % vocab.vocab_size)

Vocab has a size of 32100



## Set the preprocessors and the task registry for the t5 model

In [3]:
def python_sourceSum_dataset_fn(split, shuffle_files=False):
    del shuffle_files

    ds = tf.data.TextLineDataset(source_sum_python_path[split])
    ds = ds.map(
        functools.partial(tf.io.decode_csv, record_defaults=["", ""], field_delim="\t", use_quote_delim=False),
        num_parallel_calls=tf.data.experimental.AUTOTUNE
    )
    ds = ds.map(lambda *ex: dict(zip(["code", "docstring"], ex)))
    return ds


def source_sum_python_preprocessor(ds):
    def normalize_text(text):
        return text

    def to_inputs_and_targets(ex):
        return {
            "inputs": tf.strings.join(["source code summarization python: ", normalize_text(ex["code"])]),
            "targets": normalize_text(ex["docstring"])
        }

    return ds.map(to_inputs_and_targets, num_parallel_calls=tf.data.experimental.AUTOTUNE)


t5.data.TaskRegistry.remove('sourcesum_python_code')
t5.data.TaskRegistry.add(
    "sourcesum_python_code",
    dataset_fn=python_sourceSum_dataset_fn,
    output_features={
        "inputs": t5.data.utils.Feature(vocabulary=vocab),
        "targets": t5.data.utils.Feature(vocabulary=vocab),
    },
    splits=["train", "validation"],
    text_preprocessor=[source_sum_python_preprocessor],
    postprocess_fn=t5.data.postprocessors.lower_text,
    metric_fns=[t5.evaluation.metrics.bleu, t5.evaluation.metrics.accuracy, t5.evaluation.metrics.rouge],
)

## Set t5 small model

In [4]:
MODEL_DIR = "small"
model_parallelism = 1
train_batch_size = 256

tf.io.gfile.makedirs(MODEL_DIR)
model = t5.models.MtfModel(
    model_dir=MODEL_DIR,
    tpu=None,
    tpu_topology=None,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size,
    sequence_length={"inputs": 512, "targets": 512},
    mesh_shape="model:1,batch:1",
    mesh_devices=["GPU:0"],
    learning_rate_schedule=0.003,
    save_checkpoints_steps=5000,
    keep_checkpoint_max=None,
    iterations_per_loop=100,
)

## Code Documentation Summarization

### Give the code for summarization

In [24]:
code = '''with open("file.txt", "r") as in_file:\n    buf = in_file.readlines()\n\nwith open("file.txt", "w") as out_file:\n    for line in buf:\n        if line == "; Include this text\n":\n            line = line + "Include below\n"\n        out_file.write(line)''' #@param {type:"raw"}


### Parsing and Tokenization



In [25]:
import tokenize
import io

def pythonTokenizer(line):
    result= []
    line = io.StringIO(line)  # for python 3 use io.StringIO
    
    for toktype, tok, start, end, line in tokenize.generate_tokens(line.readline):
        # we can also use token.tok_name[toktype] instead of 'COMMENT'
        # from the token module
        if (not toktype == tokenize.COMMENT):
            if toktype == tokenize.STRING:
                result.append("CODE_STRING")
            elif toktype == tokenize.NUMBER:
                result.append("CODE_INTEGER")
            elif (not tok=="\n") and (not tok=="    "):
                result.append(str(tok))
    return ' '.join(result)

In [26]:
tokenized_code = pythonTokenizer(code)
print("code after tokenization " + tokenized_code)

code after tokenization with open ( CODE_STRING , CODE_STRING ) as in_file : buf = in_file . readlines ( )  with open ( CODE_STRING , CODE_STRING ) as out_file : for line in buf :          if line ==   " ; Include this text   " :              line = line +   " Include below  "          out_file . write ( line )   


### Record the code for summarization with the prefix to a txt file

In [27]:
codes = [tokenized_code]

inputs_path = 'input.txt'
with tf.io.gfile.GFile(inputs_path, "w") as f:
  for c in codes:
    f.write("source code summarization python: %s\n" % c)

predict_outputs_path = 'MtfModel-output.txt'


### Running the model with the best checkpoint to summarize the given code

In [28]:
model.batch_size = 8  # Min size for small model on v2-8 with parallelism 1.
model.predict(
    input_file="input.txt",
    output_file=predict_outputs_path,
    checkpoint_steps=5000,
    beam_size=4,
    vocabulary=vocab, 
    # Select the most probable output token at each step.
    temperature=0,
)

INFO:tensorflow:Using config: {'_model_dir': 'small', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=100, num_shards=None, num_cores_per_replica=1, per_host_input_for_training=4, tpu_job_name=None, initial_infeed_sleep_s

### Code Summarization Result

In [29]:
prediction_file = "MtfModel-output.txt-5000"
print("\nPredictions using checkpoint 5000:\n" )
with tf.io.gfile.GFile(prediction_file) as f:
  for c, d in zip(codes, f):
    if c:
      print("Code for prediction: " + c + '\n')
      print("Generated Summarization: " + d)



Predictions using checkpoint 5000:

Code for prediction: with open ( CODE_STRING , CODE_STRING ) as in_file : buf = in_file . readlines ( )  with open ( CODE_STRING , CODE_STRING ) as out_file : for line in buf :          if line ==   " ; Include this text   " :              line = line +   " Include below  "          out_file . write ( line )   

Generated Summarization: b'Python : Compare files containing a folder'

